In [3]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import torch
from annoy import AnnoyIndex
import forward_ranker.load_data as load_data
import forward_ranker.train as train
from forward_ranker.utils import print_message
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer
import pickle
import numpy as np

In [6]:
query_test_emb = obj_reader("/home/jianx/results/test_query_0__emb_p__data_obj_0.pb")
query_test_embid = obj_reader("/home/jianx/results/test_query_0__embid_p__data_obj_0.pb")

In [11]:
qid_test_offset = obj_reader("/datadrive/data/preprocessed_data_with_test/test-query_qid2offset.pickle")

In [ ]:
with open("/home/jianx/results/dev_query_0__emb_p__data_obj_0.pb", 'rb') as handle:
    query_dev_emb = pickle.load(handle, encoding="bytes")
with open("/home/jianx/results/dev_query_0__embid_p__data_obj_0.pb", 'rb') as handle:
    query_dev_embid = pickle.load(handle, encoding="bytes")


def generate_annoy_index(embed_size, embeddings, raw_data_path):
    embedding_dict = {}
    with open(raw_data_path, "r") as f:
        rc = 0
        for line in f:
            embedding_dict[int(line.split()[0])] = embeddings[rc]
            rc += 1
    mapping = {}
    i = 0
    index = AnnoyIndex(embed_size, 'dot')
    for key, value in embedding_dict.items():
        index.add_item(i, value)
        mapping[i] = key
        i += 1
        if i % 50000 == 0:
            print_message("Progress: " + str(i) + "/" + str(len(embedding_dict)) + " " + str(i / len(embeddings)))
    return index, mapping

query_dev_index, query_dev_mapping = generate_annoy_index(768, query_dev_emb, "/datadrive/data/raw_data/queries.dev.small.tsv")
print_message("Start Building.")
query_dev_index.build(100)
print_message("Finished Building.")

query_dev_index.save("/datadrive/jianx/data/annoy/" + str(100) + "_ance_query_dev_index.ann")
obj_writer(query_dev_mapping, "/datadrive/jianx/data/annoy/" + str(100) + "_ance_query_dev_map.dict")
print_message("Successfully Saved.")

In [21]:
query_dev_dict = {}
with open("/datadrive/data/raw_data/queries.dev.small.tsv", "r") as f:
        rc = 0
        for line in f:
            query_dev_dict[rc] = line.split("\t")[1]
            rc += 1
nearest = query_dev_index.get_nns_by_item(2, 1000)
for annoy_id in nearest:
    print(query_dev_dict[annoy_id])

treating tension headaches without medication

average wage gilded age

what are the three monetary policy tools of the fed

what is the best merv rating for filters?

what does human enhancement mean

where was the mlk speech given

green card definition

how do utis happen

what is the best medication for hemorrhoids

what nationality is sanders

what is sunchoke

what sickness has headache and nausea symptoms

is hsa account taxable

what does it mean when you dream about babies

what is pharmaceutical/medical waste

how do you call a rapid response in the hospital?

how much garlic powder replaces a clove of garlic

what is michael strahans salary

who has absolute monarchy

how tall was hulk hogan in his prime

when do you use anova

is glycogen stored in the liver

what does address label mean

where in the genes is the disorder located

when did president obama tickets first vacation while in office

temperature for home freezer

what wages does a police officer and a sheriff ne

In [4]:
passage_embeddings = obj_reader("/home/jianx/results/passage_0__emb_p__data_obj_0.pb")
print(passage_embeddings.shape)

(8841823, 768)


In [ ]:
passage_index, passage_mapping = generate_annoy_index(768, passage_embeddings, "/datadrive/data/raw_data/collection.tsv")
print_message("Start Building.")
passage_index.build(100)
print_message("Finished Building.")

[Aug 01, 19:25:43] Progress: 50000/8841823 0.005654942425334685
[Aug 01, 19:25:49] Progress: 100000/8841823 0.01130988485066937
[Aug 01, 19:25:55] Progress: 150000/8841823 0.016964827276004055
[Aug 01, 19:26:00] Progress: 200000/8841823 0.02261976970133874
[Aug 01, 19:26:06] Progress: 250000/8841823 0.028274712126673426
[Aug 01, 19:26:11] Progress: 300000/8841823 0.03392965455200811
[Aug 01, 19:26:17] Progress: 350000/8841823 0.0395845969773428
[Aug 01, 19:26:22] Progress: 400000/8841823 0.04523953940267748
[Aug 01, 19:26:28] Progress: 450000/8841823 0.050894481828012164
[Aug 01, 19:26:34] Progress: 500000/8841823 0.05654942425334685
[Aug 01, 19:26:39] Progress: 550000/8841823 0.06220436667868153
[Aug 01, 19:26:45] Progress: 600000/8841823 0.06785930910401622
[Aug 01, 19:26:51] Progress: 650000/8841823 0.0735142515293509
[Aug 01, 19:26:56] Progress: 700000/8841823 0.0791691939546856
[Aug 01, 19:27:02] Progress: 750000/8841823 0.08482413638002027
[Aug 01, 19:27:07] Progress: 800000/8841

In [1]:
passage_index.save("/datadrive/jianx/data/annoy/" + str(100) + "_ance_passage_index.ann")
obj_writer(passage_mapping, "/datadrive/jianx/data/annoy/" + str(100) + "_ance_passage_map.dict")
print_message("Successfully Saved.")

NameError: name 'passage_index' is not defined

In [13]:
all_train_qids = set()
with open("/datadrive/data/raw_data/queries.train.tsv") as f:
    for line in f:
        all_train_qids.add(int(line.split("\t")[0]))
print(len(all_train_qids))
train_qids = []
with open("/datadrive/data/raw_data/qrels.train.tsv") as f:
    for line in f:
        qid = int(line.split("\t")[0])
        if qid in all_train_qids:
            train_qids.append(qid)

print(len(train_qids))

808731
532761


In [25]:
qid_offset = obj_reader("/datadrive/data/preprocessed_data_with_test/test-query_qid2offset.pickle")
mapping = {v: k for k, v in qid_offset.items()}

print(mapping)
print(qid_offset)

{0: 47923, 1: 1124210, 2: 130510, 3: 405717, 4: 490595, 5: 146187, 6: 1103812, 7: 131843, 8: 1037798, 9: 104861, 10: 915593, 11: 833860, 12: 489204, 13: 264014, 14: 207786, 15: 1121402, 16: 87181, 17: 573724, 18: 962179, 19: 1117099, 20: 182539, 21: 443396, 22: 168216, 23: 148538, 24: 1114819, 25: 156493, 26: 451602, 27: 1121709, 28: 1110199, 29: 359349, 30: 87452, 31: 1133167, 32: 1115776, 33: 183378, 34: 1063750, 35: 1113437, 36: 1112341, 37: 855410, 38: 1106007, 39: 1114646, 40: 1129237, 41: 19335, 42: 527433}
{47923: 0, 1124210: 1, 130510: 2, 405717: 3, 490595: 4, 146187: 5, 1103812: 6, 131843: 7, 1037798: 8, 104861: 9, 915593: 10, 833860: 11, 489204: 12, 264014: 13, 207786: 14, 1121402: 15, 87181: 16, 573724: 17, 962179: 18, 1117099: 19, 182539: 20, 443396: 21, 168216: 22, 148538: 23, 1114819: 24, 156493: 25, 451602: 26, 1121709: 27, 1110199: 28, 359349: 29, 87452: 30, 1133167: 31, 1115776: 32, 183378: 33, 1063750: 34, 1113437: 35, 1112341: 36, 855410: 37, 1106007: 38, 1114646: 39

In [26]:
passage_index = AnnoyIndex(768, 'dot')
passage_index.load("/datadrive/jianx/data/annoy/100_ance_passage_index.ann")

In [27]:
pid_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_passage_map.dict")

In [38]:
qid_test_mapping = {v: k for k, v in qid_test_offset.items()}
result_dict = {}

for idx, embedding in enumerate(query_test_emb):
    top_list = passage_index.get_nns_by_vector(embedding, 1000)
    print(top_list)
    qid = qid_test_mapping[idx]
    for j, index in enumerate(top_list):
        if index in pid_mapping:
            top_list[j] = pid_mapping[index]
    q_results = {}
    score = float(len(top_list))
    for j, pid in enumerate(top_list):
        q_results[pid] = score
        score -= 1
    result_dict[qid] = q_results

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [37]:
print(result_dict)

{47923: {}, 1124210: {}, 130510: {}, 405717: {}, 490595: {}, 146187: {}, 1103812: {}, 131843: {}, 1037798: {}, 104861: {}, 915593: {}, 833860: {}, 489204: {}, 264014: {}, 207786: {}, 1121402: {}, 87181: {}, 573724: {}, 962179: {}, 1117099: {}, 182539: {}, 443396: {}, 168216: {}, 148538: {}, 1114819: {}, 156493: {}, 451602: {}, 1121709: {}, 1110199: {}, 359349: {}, 87452: {}, 1133167: {}, 1115776: {}, 183378: {}, 1063750: {}, 1113437: {}, 1112341: {}, 855410: {}, 1106007: {}, 1114646: {}, 1129237: {}, 19335: {}, 527433: {}}


In [13]:
def get_ndcg_precision_rr(true_dict, test_dict, rank):
    sorted_result = sorted(test_dict.items(), key=lambda x: (x[1], [-1,1][random.randrange(2)]), reverse=True)
    original_rank = rank
    rank = min(rank, len(sorted_result))
    cumulative_gain = 0
    num_positive = 0
    rr = float("NaN")
    for i in range(len(sorted_result)):
        pid = sorted_result[i][0]
        if pid in true_dict:
            rr = 1 / (i + 1)
            break
    for i in range(rank):
        pid = sorted_result[i][0]
        if pid in true_dict:
            num_positive += 1
    sorted_result = sorted(test_dict.items(), key=lambda x: x[1], reverse=True)
    for i in range(rank):
        pid = sorted_result[i][0]
        relevance = 0
        if pid in true_dict:
            relevance = true_dict[pid]
        discounted_gain = relevance / math.log2(2 + i)
        cumulative_gain += discounted_gain
    sorted_ideal = sorted(true_dict.items(), key=lambda x: x[1], reverse=True)
    ideal_gain = 0
    for i in range(rank):
        relevance = 0
        if i < len(sorted_ideal):
            relevance = sorted_ideal[i][1]
        discounted_gain = relevance / math.log2(2 + i)
        ideal_gain += discounted_gain
    ndcg = 0
    if ideal_gain != 0:
         ndcg = cumulative_gain / ideal_gain
    return ndcg, num_positive / original_rank, rr

In [36]:
import numpy as np
rating_dict = obj_reader("/datadrive/jianx/data/rel_scores.dict")
rank = 10

qids = list(result_dict.keys())
result_ndcg = []
result_prec = []
result_rr = []
for qid in qids:
    if qid in rating_dict:
        ndcg, prec, rr = get_ndcg_precision_rr(rating_dict[qid], result_dict[qid], rank)
        result_ndcg.append(ndcg)
        result_prec.append(prec)
        result_rr.append(rr)
avg_ndcg = np.nanmean(result_ndcg)
avg_prec = np.nanmean(result_prec)
avg_rr = np.nanmean(result_rr)
print(avg_ndcg, avg_prec, avg_rr)

0.0 0.0 nan


/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice


In [29]:
raw_passages = {}
with open("/datadrive/jianx/data/collection.tsv") as f:
    for line in f:
        raw_passages[int(line.split("\t")[0])] = line.split("\t")[1]

In [44]:
raw_query_train = {}
with open("/datadrive/jianx/data/queries.train.tsv") as f:
    for line in f:
        raw_query_train[int(line.split("\t")[0])] = line.split("\t")[1]

In [53]:
query_train_index = AnnoyIndex(768, 'dot')
query_train_index.load("/datadrive/jianx/data/annoy/100_query_train_ance_index.ann")
query_train_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_query_train_map.dict")

ID = 175960
print(raw_passages[ID])
for i, index in enumerate(query_train_index.get_nns_by_vector(passage_embeddings[ID], 1000)):
    print(query_train_mapping[index], raw_query_train[query_train_mapping[index]])

Diamond Head is an extinct volcano and a popular visitor attraction on Oahu. You can hike up to its 760-foot-high peak, but be prepared to climb stairs. Amazing views of Honolulu, Waikiki and the vast ocean await at the top; on clear days the neighbor islands of Molokai and Lanai can be seen on the horizon.

180151 employee burden definition

118600 define burden tax

130283 definition burdened cost

407929 is defined as practices that are inconsistent with sound fiscal, business, or medical practices resulting in unnecessary costs and improper payment.

809767 what is the burden on labor

690006 what is a mandate

1168995 burden definitions

1143846 what's an undue burden

770569 what is medicare limiting charge

702306 what is a surcharge?

711513 what is an allowable cost

97432 construction costs considered a finance charge

191678 freight charges definition

691836 what is a municipal charge

622591 what do commitment charges mean

844229 what is the responsibility of a medical bi

In [5]:
import faiss
dim = passage_embeddings.shape[1]
cpu_index = faiss.IndexFlatIP(dim)
cpu_index.add(passage_embeddings)

In [7]:
_, dev_I = cpu_index.search(query_test_emb, 1000)

In [19]:
qid_test_mapping = {v: k for k, v in qid_test_offset.items()}
result_dict = {}

for idx, top_list in enumerate(dev_I):
    qid = qid_test_mapping[idx]
    for j, index in enumerate(top_list):
        if index in pid_mapping:
            top_list[j] = pid_mapping[index]
    q_results = {}
    score = float(len(top_list))
    for j, pid in enumerate(top_list):
        q_results[pid] = score
        score -= 1
    result_dict[qid] = q_results
print(result_dict)

{47923: {618556: 1000.0, 6241150: 999.0, 4297620: 998.0, 6241151: 997.0, 5417582: 996.0, 6241152: 995.0, 1681332: 994.0, 6241147: 993.0, 8418681: 992.0, 1681334: 991.0, 8418683: 990.0, 5417578: 989.0, 6241148: 988.0, 8641107: 987.0, 8641104: 986.0, 6643066: 985.0, 5417581: 984.0, 5417579: 983.0, 6241154: 982.0, 3008962: 981.0, 473807: 980.0, 8327407: 979.0, 6191: 978.0, 913042: 977.0, 8418686: 976.0, 3008961: 975.0, 566677: 974.0, 3008963: 973.0, 1681336: 972.0, 385678: 971.0, 459890: 970.0, 1681339: 969.0, 2139524: 968.0, 951595: 967.0, 385679: 966.0, 4286048: 965.0, 986177: 964.0, 2139522: 963.0, 8421421: 962.0, 459887: 961.0, 388874: 960.0, 2139523: 959.0, 951591: 958.0, 385680: 957.0, 288290: 956.0, 6241153: 955.0, 6960364: 954.0, 1681331: 953.0, 966946: 952.0, 5417577: 951.0, 4436801: 950.0, 4814607: 949.0, 3744477: 948.0, 459886: 947.0, 951592: 946.0, 388877: 945.0, 2699737: 944.0, 5147526: 943.0, 913049: 942.0, 6960365: 941.0, 966951: 940.0, 1681337: 939.0, 966945: 938.0, 594968

In [22]:
import numpy as np
import random
import math
rating_dict = obj_reader("/datadrive/jianx/data/rel_scores.dict")
rank = 10

qids = list(result_dict.keys())
result_ndcg = []
result_prec = []
result_rr = []
for qid in qids:
    if qid in rating_dict:
        ndcg, prec, rr = get_ndcg_precision_rr(rating_dict[qid], result_dict[qid], rank)
        result_ndcg.append(ndcg)
        result_prec.append(prec)
        result_rr.append(rr)
avg_ndcg = np.nanmean(result_ndcg)
avg_prec = np.nanmean(result_prec)
avg_rr = np.nanmean(result_rr)
print(avg_ndcg, avg_prec, avg_rr)

0.5880055767930913 0.6558139534883722 0.8712641080542516


In [44]:
print(qids)

[47923, 1124210, 130510, 405717, 490595, 146187, 1103812, 131843, 1037798, 104861, 915593, 833860, 489204, 264014, 207786, 1121402, 87181, 573724, 962179, 1117099, 182539, 443396, 168216, 148538, 1114819, 156493, 451602, 1121709, 1110199, 359349, 87452, 1133167, 1115776, 183378, 1063750, 1113437, 1112341, 855410, 1106007, 1114646, 1129237, 19335, 527433]


In [45]:
ID = 146187

raw_query_test = {}
with open("/datadrive/jianx/data/msmarco-test2019-queries.tsv") as f:
    for line in f:
        raw_query_test[int(line.split("\t")[0])] = line.split("\t")[1]
print(raw_query_test[ID])

for index in result_dict[ID]:
    print(index, raw_passages[index])

difference between a mcdouble and a double cheeseburger

8434623 There is a discernible difference between the two burgers, especially in the meat quality. The McDouble contains gristle and carry's a different flavor than the Double Cheeseburger. The double cheeseburger tastes more like beef. The McDouble tastes of whatever meat filler they use to fill the gap.

8434625 A double cheesburger has 2 slices of cheese with 2 meat patties. A mcdouble on the other hand only had 1 slice of cheese, but still 2 pieces of meat. I say had because mcdoubles arent sold in Australia (maybe a few stores but not a regular menu item) anymore. Other countries might be different but that's the difference in Australia.

8434619 At first glance, the double cheeseburger looks a lot like the McDouble, but it is not exactly the same thing. The main difference that you will notice is the amount of cheese that is on this sandwich. There are two slices of cheese on the double cheeseburger instead of just the one 